In [ ]:
#Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import tan, radians
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing

lm = LinearRegression()
standard_scaler = preprocessing.StandardScaler()

In [ ]:
def standardize_data(x):

    features = []

    #identify indicies
    indexControl = np.where(x[0]== 'Control')
    indexAttention = np.where(x[0]== 'Attention')
    indexTangibles = np.where(x[0]== 'Tangibles')
    indexEscape = np.where(x[0]== 'Escape')
    indexAlone = np.where(x[0]== 'Alone')

    #extract data
    Control = (x[1][indexControl].astype(float)).flatten()
    Attention = (x[1][indexAttention].astype(float)).flatten()
    Tangibles = (x[1][indexTangibles].astype(float)).flatten()
    Escape = (x[1][indexEscape].astype(float)).flatten()
    Alone = (x[1][indexAlone].astype(float)).flatten()

    #mean and SD
    overallMean = np.mean(np.hstack((Control, Attention, Tangibles, Escape, Alone)))
    overallSd = np.std(np.hstack((Control, Attention, Tangibles, Escape, Alone)))

    #standardize
    Control  = (Control-overallMean)/overallSd
    Control[np.isnan(Control)] = 0

    Attention = (Attention-overallMean)/overallSd
    Attention[np.isnan(Attention)] = 0

    Tangibles = (Tangibles-overallMean)/overallSd
    Tangibles[np.isnan(Tangibles)] = 0

    Escape = (Escape-overallMean)/overallSd
    Escape[np.isnan(Escape)] = 0

    Alone = (Alone-overallMean)/overallSd
    Alone[np.isnan(Alone)] = 0

    pointsControl = len(Control)
    pointsTotal = pointsControl + len(Attention) + len(Tangibles) + len(Escape) + len(Alone)

    #append
    features.append(np.mean(Control))
    features.append(np.mean(Attention))
    features.append(np.mean(Tangibles))
    features.append(np.mean(Escape))
    features.append(np.mean(Alone))
    features.append(np.std(Control))
    features.append(np.std(Attention))
    features.append(np.std(Tangibles))
    features.append(np.std(Escape))
    features.append(np.std(Alone))

    #lr
    lm1 = LinearRegression().fit(np.array(range(pointsControl)).reshape(-1,1),
                    np.expand_dims(Control, axis =1))
    features.append(float(lm1.intercept_))
    features.append(float(lm1.coef_))

    lm2 = LinearRegression().fit(np.array(range(len(Attention))).reshape(-1,1),
                    np.expand_dims(Attention, axis =1))
    features.append(float(lm2.intercept_))
    features.append(float(lm2.coef_))

    lm3 = LinearRegression().fit(np.array(range(len(Tangibles))).reshape(-1,1),
                    np.expand_dims(Tangibles, axis =1))
    features.append(float(lm3.intercept_))
    features.append(float(lm3.coef_))

    lm4 = LinearRegression().fit(np.array(range(len(Escape))).reshape(-1,1),
                    np.expand_dims(Escape, axis =1))
    features.append(float(lm4.intercept_))
    features.append(float(lm4.coef_))

    lm5 = LinearRegression().fit(np.array(range(len(Alone))).reshape(-1,1),
                    np.expand_dims(Alone, axis =1))
    features.append(float(lm5.intercept_))
    features.append(float(lm5.coef_))

    features = np.array(features).flatten()

    return features

In [ ]:
#Call
all_ME_data, true_values, trend_values = perform_simulations()

#Collect all features
all_features = [standardize_data(ME_data) for ME_data in all_ME_data]

#Convert to DataFrame
columns = [
    'Mean Control', 'Mean Attention', 'Mean Tangibles', 'Mean Escape', 'Mean Alone',
    'Std Control', 'Std Attention', 'Std Tangibles', 'Std Escape', 'Std Alone',
    'Intercept Control', 'Slope Control',
    'Intercept Attention', 'Slope Attention',
    'Intercept Tangibles', 'Slope Tangibles',
    'Intercept Escape', 'Slope Escape',
    'Intercept Alone', 'Slope Alone'
]

df_features = pd.DataFrame(all_features, columns=columns)

#Save DataFrame to CSV
df_features.to_csv('std_features_5condition.csv', index=False)

print("Features saved to std_features_5condition.csv")